In [97]:
####################################################################################################
#  This will read in a model number and output the stats for that specific model.
# It will calculate the KGE (Kling-Gupta Efficiency), RMSE, and Residuals for the model.
# However, given that some of these models have different regional models that make up the main model,
# it will need to be separated into different regions. But only for the models that do have regional models.
# The stats will be outputted to a csv file. To be used in the future for analysis.
####################################################################################################

import sys
import re
import numpy as np
import pandas as pd
import geopandas as gpd

In [98]:
# Caclulate the RMSE for a predicted and observed dataset
# Assume that the observed and predicted datasets are pandas dataframes with the same length
def rmse(observed, predicted):
    return np.sqrt(np.mean((observed - predicted) ** 2))

In [99]:
# Calculate the KGE for a predicted and observed dataset
# Assume that the observed and predicted datasets are pandas dataframes with the same length
def kge(observed, predicted):
    # Calculate mean and standard deviation of observed and predicted datasets
    obs_mean = observed.mean()
    sim_mean = predicted.mean()
    obs_std = observed.std()
    sim_std = predicted.std()

    # Calculate the correlation coefficient between the observed and predicted datasets
    r = np.corrcoef(observed, predicted)[0, 1] # [0, 1] is the correlation between the two datasets in the matrix

    # Calculate the bias and variability between the observed and predicted datasets
    beta = sim_mean / obs_mean
    alpha = sim_std / obs_std

    # Calculate the Kling-Gupta Efficiency
    return 1 - np.sqrt((r - 1) ** 2 + (alpha - 1) ** 2 + (beta - 1) ** 2)

In [122]:
# Create a function that will load in the model data and return a geopandas dataframe with the data and
# the model architecture
def readModelData():
    # This reads what the user inputs for the model number and the architecture used
    modelNum = 1#sys.argv[1]
    modelArch = "Global"#"PrevailingWInds_6Split"#sys.argv[2]

    # Load in the model data
    modelData = pd.read_csv(f"../Models/Model_{modelNum}/Model_{modelNum}_TestData.csv")
    modelData = gpd.GeoDataFrame(modelData, geometry=gpd.points_from_xy(modelData.Lon, modelData.Lat), crs="EPSG:4326")

    # Change the headers of the columns to be more usable with the code
    # Start by extracting the old headers
    oldHeaders = modelData.columns

    # Now go through and change the headers to be more usable using regex to remove anything within brackets
    # Also if there is something like O18 Actual, it will be changed to O18A and O18P for the predicted
    newHeaders = []
    newHeaders = [re.sub(r"\(.*\)", "", header).strip() for header in oldHeaders]
    newHeaders = [re.sub(r" Actual", "A", header).strip() for header in newHeaders]
    newHeaders = [re.sub(r" Predicted", "P", header).strip() for header in newHeaders]

    # Now change the headers of the dataframe
    modelData.columns = newHeaders
    

    # Load in the model architecture which will only need to be done if not "Global"
    if modelArch != "Global":
        modelArch = pd.read_csv(f"../Data/ModelSplit_Arch/{modelArch}.csv")
        modelArch = gpd.GeoDataFrame(modelArch, geometry=gpd.GeoSeries.from_wkt(modelArch.geometry), crs="EPSG:4326")
    else:
        modelArch = None

    return modelData, modelArch, list(oldHeaders)

In [120]:
# Calculate the summary statistics for the model and return a dataframe with the results for each region
# and if there are no regions, return the stats for the entire model.
def summaryStats(modelData, modelArch):
    # First check if the model has regional models
    if modelArch is not None:
        # Cycle through the regions and assign the data from each to dictionary
        regionalData = {}
        key = modelArch.columns[0]
        for index, region in modelArch.iterrows():
            regionalData[region[key]] = modelData[modelData.within(region.geometry)]
        
        # Calculate the stats for each region and store in a dictionary
        regionalStats = pd.DataFrame(columns=["Region", "O18 KGE", "O18 RMSE", "H2 KGE", "H2 RMSE"])
        for region, data in regionalData.items():
            stats = {"O18 KGE": kge(data['O18 A'], data['O18 P']),
                     "O18 RMSE": rmse(data['O18 A'], data['O18 P']),
                     "H2 KGE": kge(data['H2 A'], data['H2 P']),
                     "H2 RMSE": rmse(data['H2 A'], data['H2 P']),}
            regionalStats = pd.concat([regionalStats, pd.DataFrame({"Region": [region], **stats})], ignore_index=True)
        
        return regionalStats
    else:
        # Calculate the stats for the entire model
        stats = {"O18 KGE": kge(modelData['O18 A'], modelData['O18 P']),
                 "O18 RMSE": rmse(modelData['O18 A'], modelData['O18 P']),
                 "H2 KGE": kge(modelData['H2 A'], modelData['H2 P']),
                 "H2 RMSE": rmse(modelData['H2 A'], modelData['H2 P']),}
        
        return pd.DataFrame([stats])

In [123]:
modelData, modelArch, oldHeaders = readModelData()
summaryStats(modelData, modelArch)

,O18 KGE,O18 RMSE,H2 KGE,H2 RMSE
0,0.839538,2.509075,0.861782,19.2546
